In [ ]:
!pip install pandas scikit-learn nltk

In [ ]:
import pandas as pd
import nltk
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from google.colab import files
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')

# Upload file CSV
uploaded = files.upload()


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Saving hasil_berita_politik.csv to hasil_berita_politik (2).csv


In [ ]:
# Membaca file CSV
file_name = list(uploaded.keys())[0]
df = pd.read_csv(file_name)

# Check the column names
print(df.columns)

# Tampilkan beberapa baris pertama dari file CSV
df.head()

# Fungsi untuk membersihkan teks
def preprocess_text(text):
    # Check if the text is a string to avoid errors with non-string values
    if isinstance(text, str):
        # Ubah teks menjadi huruf kecil
        text = text.lower()

        # Menghapus tanda baca
        text = text.translate(str.maketrans('', '', string.punctuation))

        # Tokenisasi (memecah teks menjadi kata-kata)
        words = word_tokenize(text)

        # Menghapus stopwords (kata-kata umum yang tidak penting)
        stop_words = set(stopwords.words('indonesian'))  # Bahasa Indonesia
        words = [word for word in words if word not in stop_words]

        return " ".join(words)
    else:
        return text  # Return the original value if it's not a string

# Access the correct column name
# Replace 'Link_Text' with the actual name
# of the column you want to process from the df.columns output
df['cleaned_text'] = df['Link_Text'].apply(preprocess_text)

# Tampilkan hasil setelah preprocessing
df[['Link_Text', 'cleaned_text']].head()

Index(['Link_Text', 'Link_URL'], dtype='object')


,Link_Text,cleaned_text
0,login,login
1,\n\n,
2,Home,home
3,\n Nasional,nasional
4,Politik,politik


In [ ]:
# Check for missing values in 'cleaned_text'
print(df['cleaned_text'].isnull().sum())

# Remove rows with missing values in 'cleaned_text' (fitur teks yang telah dibersihkan)
df = df.dropna(subset=['cleaned_text'])

# Check the available columns
print(df.columns)

# Ensure 'label' or the correct target column exists for classification task
# Replace 'label' with the correct target column name from df.columns
# Assume the column 'label' is your target variable (which might be a categorical class)
y = df['Link_URL']  # Replace 'label' with the actual column name that contains your labels

# Generate the TF-IDF matrix from the 'cleaned_text' column
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()
X = tfidf.fit_transform(df['cleaned_text'])

# Split data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Modeling: Using Logistic Regression (Simple Model)
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)

# Print accuracy score (for example)
from sklearn.metrics import accuracy_score
print("Accuracy:", accuracy_score(y_test, y_pred))


0
Index(['Link_Text', 'Link_URL', 'cleaned_text'], dtype='object')
Accuracy: 0.22580645161290322


In [ ]:
# Inisialisasi TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer()

# Menerapkan TF-IDF pada kolom teks yang sudah dipreprocessing
# Fill NaN values with empty strings to avoid errors
tfidf_matrix = tfidf_vectorizer.fit_transform(df['Link_Text'].fillna(''))

# Menampilkan fitur-fitur (kata-kata unik)
tfidf_features = tfidf_vectorizer.get_feature_names_out()

# Konversi hasil TF-IDF ke dalam DataFrame untuk dilihat lebih jelas
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_features)

# Menampilkan hasil TF-IDF
tfidf_df.head()

,00,01,02,03,04,05,06,10,12,13,...,vs,wantimpres,warga,wib,witan,wni,yang,yaqut,العربية,日本語
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Menghitung cosine similarity antara dokumen
cosine_sim = cosine_similarity(tfidf_matrix)

# Menampilkan hasil cosine similarity
cosine_sim_df = pd.DataFrame(cosine_sim)

# Menampilkan beberapa baris dari matriks cosine similarity
cosine_sim_df.head()


,0,1,2,3,4,5,6,7,8,9,...,145,146,147,148,149,150,151,152,153,154
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.707107,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Simpan hasil TF-IDF ke file CSV
tfidf_df.to_csv('Politik_tfidf_output.csv', index=False)

# Simpan hasil cosine similarity ke file CSV
cosine_sim_df.to_csv('Politik_cosine_similarity.csv', index=False)

# Download file CSV
files.download('Politik_tfidf_output.csv')
files.download('Politik_cosine_similarity.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>